In [23]:
import sys
import numpy as np
import os
current_directory = os.getcwd()
relative_path1 = '../build/env_simulator'
relative_path2 = '../build/cilqr'
file_path1 = os.path.join(current_directory, relative_path1)
file_path2 = os.path.join(current_directory, relative_path2)
sys.path.append(file_path1)
sys.path.append(file_path2)
relative_path3 = '../env_simulator'
file_path3 = os.path.join(current_directory, relative_path3)
sys.path.append(file_path3)
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import pybind_ilqr
import pybind_env_simulator
from py_lib import py_env_sim
from py_lib import cilqr_plot
from bokeh.models import ColumnDataSource
from bokeh.models import WheelZoomTool
from bokeh.models import HoverTool
from pprint import *
from bokeh.layouts import column, row
from bokeh.models import HoverTool
from bokeh.models import LinearColorMapper, ColorBar, ColumnDataSource, HoverTool
from bokeh.transform import linear_cmap
from bokeh.palettes import Viridis
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import Viridis256
from bokeh.transform import linear_cmap
import numpy as np
import math

In [24]:
idm_params = pybind_ilqr.IDMParam()
idm_params.delta = 3.0
idm_params.acc_delta = 2.0
idm_params.max_acc = 1.2
idm_params.time_headway = 1.0
idm_params.follow_car_delta = 1.2
idm_params.min_dacc = 3.5
idm_params.v_std = [9.49, 18.98, 28.48]
idm_params.vel_error_std = [1.88, 2.7, 3.7]
idm_params.idm_delta_v = [0.0, 2.78, 11.11, 22.22, 90.0]
idm_params.idm_min_spacing_list = [2.0, 2.0, 2.0, 2.5, 2.5]
idm_params.delta_v = [-3.0, -2.0, -1.0, 0.0]
idm_params.s_change_rate = [0.3, 0.5, 0.7, 0.9]
idm_params.desired_spd = 0.0

In [25]:

def plot_with_color_mapping(x_values, y_values, color_values, plot_width=400, plot_height=300):
    # 创建 Bokeh 图形对象
    p = figure(title='Leader_S-Leader_V', x_axis_label='Leader_S', y_axis_label='Delta_V',
               plot_width=plot_width, plot_height=plot_height)

    # 创建 ColumnDataSource 对象
    source = ColumnDataSource(data=dict(index=range(len(x_values)), x=x_values, y=y_values, color=color_values))

    # 创建颜色映射
    mapper = linear_cmap(field_name='color', palette=Viridis[256], low=min(color_values), high=max(color_values))

    # 绘制散点图，并映射颜色
    scatter = p.scatter(x='x', y='y', size=10, color=mapper, source=source)

    # 添加颜色条
    color_bar = ColorBar(color_mapper=mapper['transform'], width=8, location=(0, 0))
    p.add_layout(color_bar, 'right')

    # 添加鼠标悬停工具
    tooltips = [("Index", "@index"), ("Leader_S", "@x"), ("Leader_V", "@y"), ("IDM_ACC", "@color")]
    hover_tool = HoverTool(renderers=[scatter], tooltips=tooltips)
    p.add_tools(hover_tool)

    # 返回图形对象
    return p

def plot_idm_acc_value_scatter(plot, x, y, dist, xx, yy, flag, x_axis_label, y_axis_label, legend_label):
    # 将数据存储在数据源中
    source = ColumnDataSource(data=dict(x=np.repeat(x, len(y)), y=np.tile(y, len(x)), dist=dist))
    # 创建颜色映射器
    color_mapper = linear_cmap(field_name='dist', palette=Viridis256, low=np.min(dist), high=np.max(dist))

    # 绘制散点图
    scatter = plot.scatter('x', 'y', radius=0.5, fill_color=color_mapper, fill_alpha=0.7, line_color=None, source=source, legend_label=legend_label)

    # 添加颜色条
    color_bar = ColorBar(color_mapper=color_mapper['transform'], width=8, location=(0, 0))
    plot.add_layout(color_bar, 'right')

    # 添加工具提示
    tooltips = [
        ("Index", "$index"),
        ("leader_s", "@x"),
        ("delta_v", "@y"),
        ("idm_acc", "@dist")
    ]
    hover_tool = HoverTool(renderers=[scatter], tooltips=tooltips)
    plot.legend.click_policy = 'hide'
    if flag:
        idm_acc_in_leader_s_xx = [x for x, acc in zip(xx, dist) if acc == -0.01]
        idm_acc_in_delta_v_yy = [y for y, acc in zip(yy, dist) if acc == -0.01]
        create_scatter_plot(plot, idm_acc_in_leader_s_xx, idm_acc_in_delta_v_yy, color='black', legend_label='IDM-ACC=-0.01')

        idm_acc_in_leader_s_xx1 = [x for x, acc in zip(xx, dist) if acc == -0.5]
        idm_acc_in_delta_v_yy1 = [y for y, acc in zip(yy, dist) if acc == -0.5]
        create_scatter_plot(plot, idm_acc_in_leader_s_xx1, idm_acc_in_delta_v_yy1, color='blue', legend_label='IDM-ACC=-0.5')

        idm_acc_in_leader_s_xx2 = [x for x, acc in zip(xx, dist) if acc == -1.0]
        idm_acc_in_delta_v_yy2 = [y for y, acc in zip(yy, dist) if acc == -1.0]
        create_scatter_plot(plot, idm_acc_in_leader_s_xx2, idm_acc_in_delta_v_yy2, color='yellow', legend_label='IDM-ACC=-1.0')

        idm_acc_in_leader_s_xx3 = [x for x, acc in zip(xx, dist) if acc == -1.5]
        idm_acc_in_delta_v_yy3 = [y for y, acc in zip(yy, dist) if acc == -1.5]
        create_scatter_plot(plot, idm_acc_in_leader_s_xx3, idm_acc_in_delta_v_yy3, color='orange', legend_label='IDM-ACC=-1.5')

        idm_acc_in_leader_s_xx4 = [x for x, acc in zip(xx, dist) if acc == -2.0]
        idm_acc_in_delta_v_yy4 = [y for y, acc in zip(yy, dist) if acc == -2.0]
        create_scatter_plot(plot, idm_acc_in_leader_s_xx4, idm_acc_in_delta_v_yy4, color='pink', legend_label='IDM-ACC=-2.0')

        idm_acc_in_leader_s_xx5 = [x for x, acc in zip(xx, dist) if acc == -3.0]
        idm_acc_in_delta_v_yy5 = [y for y, acc in zip(yy, dist) if acc == -3.0]
        create_scatter_plot(plot, idm_acc_in_leader_s_xx5, idm_acc_in_delta_v_yy5, color='green', legend_label='IDM-ACC=-3.0')

        idm_acc_in_leader_s_xx6 = [x for x, acc in zip(xx, dist) if acc == -3.5]
        idm_acc_in_delta_v_yy6 = [y for y, acc in zip(yy, dist) if acc == -3.5]
        create_scatter_plot(plot, idm_acc_in_leader_s_xx6, idm_acc_in_delta_v_yy6, color='red', legend_label='IDM-ACC=-3.5')
    else:
        idm_acc_in_leader_s_xx = [x for x, acc in zip(xx, dist) if acc >= 0.1]
        idm_acc_in_delta_v_yy = [y for y, acc in zip(yy, dist) if acc >= 0.1]
        create_scatter_plot(plot, idm_acc_in_leader_s_xx, idm_acc_in_delta_v_yy, color='black', legend_label='ACC-ERROR=0.00')

    plot.add_tools(hover_tool)

def create_scatter_plot(plot, x_values, y_values, color, legend_label):
    source = ColumnDataSource(data=dict(x=x_values, y=y_values))
    if len(y_values) == 0:
        color_mapper = 'purple'
    else:
        color_mapper = linear_cmap(field_name='y', palette=Viridis256, low=np.min(y_values), high=np.max(y_values))
    
    # 添加散点图
    scatter = plot.circle('x', 'y', fill_color=color_mapper, line_color=color, legend_label=legend_label, source=source)
    
    # 添加线图
    line = plot.line(x_values, y_values, line_color=color, line_alpha=0.8, legend_label=legend_label)
    
    # 添加悬停工具
    hover = HoverTool(renderers=[scatter], 
                      tooltips=[("Index", "$index"), 
                                ("Value", "($x, $y)")])
    if len(y_values) > 0:
        color_bar = ColorBar(color_mapper=color_mapper['transform'], width=8, location=(0, 0))
    # plot.add_layout(color_bar, 'right')
    plot.add_tools(hover)

def generate_arithmetic_list(first_element, difference, count):
    return [first_element + i * difference for i in range(count)]



## 自车80kph，期望车速80kph，leader_s和leader_v分别从10开始采样

In [26]:
leader_s = generate_arithmetic_list(10, 1, 100)  # leader_s
leader_v = generate_arithmetic_list(10, 1, 100)  # leader_v
cur_v = generate_arithmetic_list(80, 0, 100)
desired_v = generate_arithmetic_list(80, 0, 100)
delta_v = [(x - y) / 3.6 for x, y in zip(cur_v, leader_v)]
xx, yy, idm_acc_list = [], [], []
acc_crash_list = []
        
for xi in leader_s:
    for yi in delta_v:
        idm_params.desired_spd = 80 / 3.6
        res = pybind_ilqr.LongitudinalOperator.calc_idm_acc(idm_params, xi, 80 / 3.6, yi, 1)
        acc_crash = (((80/3.6) - (yi / 3.6) ) ** 2 - (80 / 3.6) ** 2) / (2 * (xi + 10))
        if (acc_crash > 0):
            acc_crash = 0.0
        idm_acc_list.append(round(res, 2))
        acc_crash_list.append(round(acc_crash, 1))
        xx.append(xi)
        yy.append(yi)
        
p1 = figure(title='Leader_S-Delta_V-ACC', plot_width=500, plot_height=500)
plot_idm_acc_value_scatter(p1, leader_s, delta_v, idm_acc_list, xx, yy, 1, 
                           x_axis_label='Leader_S(m)', y_axis_label='Delta_V(m/s)', legend_label='IDM-ACC')

def subtract_lists(list1, list2):
    result = []
    for i in range(len(list1)):
        result.append(list1[i] - list2[i])
    return result
acc_error = subtract_lists(idm_acc_list, acc_crash_list)
p2 = figure(title='IDM-ACC_Crash-ACC_ERROR', plot_width=500, plot_height=500)
plot_idm_acc_value_scatter(p2, leader_s, delta_v, acc_error, xx, yy, 0, 
                           x_axis_label='Leader_S(m)', y_axis_label='Delta_V(m/s)', legend_label='IDM-ACC_Crash-ACC_ERROR')

layout = row(p0, p1, p2)
output_notebook()
show(layout)


Loading BokehJS ...

## 无车场景下的加减速

In [27]:

cur_v = [20] * 10
acc_desired_v = generate_arithmetic_list(20, 1, 10) # (first_element, difference, count)
idm_acc_list = []
# print("------加速场景------")
for i in range(len(cur_v)):
    idm_params.desired_spd = acc_desired_v[i] / 3.6
    res = pybind_ilqr.LongitudinalOperator.calc_idm_acc(idm_params, 20, 
                                                        cur_v[i] / 3.6, 0.0, 0)
    idm_acc_list.append(res)
    # print(" no car, ", cur_v[i], " -> ", desired_v[i], " res:", res)
    # print("-------------------------------")
# pprint(idm_acc_list)
plot1 = figure(title='加速场景,自车匀速20kph', x_axis_label='desired_spd', y_axis_label='idm_acc',
                       plot_width=400, plot_height=300)
create_scatter_plot(plot1, acc_desired_v, idm_acc_list, color='blue', legend_label='Desired_Spd-Idm_Acc')

cur_v = [80] * 10
dcc_desired_v = generate_arithmetic_list(80, -2, 10) # (first_element, difference, count)
idm_dcc_list = []
# print("------减速场景------")
for i in range(len(cur_v)):
    idm_params.desired_spd = dcc_desired_v[i] / 3.6
    res = pybind_ilqr.LongitudinalOperator.calc_idm_acc(idm_params, 20, 
                                                        cur_v[i] / 3.6, 0.0, 0)
    idm_dcc_list.append(res)
    # print(" no car, ", cur_v[i], " -> ", desired_v[i], " res:", res)
    # print("-------------------------------")
# pprint(idm_dcc_list)

plot2 = figure(title='减速场景,自车匀速80kph', x_axis_label='desired_spd', y_axis_label='idm_acc',
                       plot_width=400, plot_height=300)
create_scatter_plot(plot2, dcc_desired_v, idm_acc_list, color='blue', legend_label='Desired_Spd-Idm_Acc')


layout = row(plot1, plot2)
output_notebook()
show(layout)


Loading BokehJS ...

## 自车80kph，期望车速80kph，leader_v 60kph， leader_s 从100米递减

In [28]:

print("------有车场景------")
cur_v = [80] * 20
desired_v = [80] * 20
obj_v = [60] * 20
delta_s = generate_arithmetic_list(100, -5, 20) # (first_element, difference, count)
delta_v_list = []
idm_acc_list = []
acc_crash_list = []
for i in range(len(cur_v)):
    idm_params.desired_spd = desired_v[i] / 3.6
    delta_v = (cur_v[i] - obj_v[i] ) / 3.6
    res = pybind_ilqr.LongitudinalOperator.calc_idm_acc(idm_params, delta_s[i], 
                                                        cur_v[i] / 3.6, delta_v, 1)
    idm_acc_list.append(res)
    delta_v_list.append(delta_v * 3.6)
    
    acc_crash = ((obj_v[i] / 3.6) ** 2 - (cur_v[i] / 3.6) ** 2) / (2 * delta_s[i])
    acc_crash_list.append(acc_crash)
    # print(" has car, ", cur_v[i], " -> desired spd is:", desired_v[i],
    #       " , obj_v is:", obj_v[i], " , delta s is:", delta_s[i], " res:", res)
    # print("-------------------------------")
# pprint(idm_acc_list)

plot1 = figure(title='Leader_S-Acc', x_axis_label='leader_s', y_axis_label='acc',
                       plot_width=400, plot_height=300)
create_scatter_plot(plot1, delta_s, idm_acc_list, color='blue', legend_label='Leader_S-Idm_Acc')
# create_scatter_plot(plot1, delta_s, acc_crash_list, color='red', legend_label='Leader_S-Crash_Acc')
plot1.legend.click_policy = 'hide'
plot2 = figure(title='Leader_V-Acc', x_axis_label='leader_v', y_axis_label='acc',
                       plot_width=400, plot_height=300)
create_scatter_plot(plot2, obj_v, idm_acc_list, color='blue', legend_label='Leader_V-Idm_Acc')
plot2.legend.click_policy = 'hide'
plot3 = plot_with_color_mapping(delta_s, delta_v_list, idm_acc_list, plot_width=400, plot_height=300)
# 将两个图横向放置在同一行
layout = row(plot1, plot2, plot3)

# 在Jupyter Notebook中显示图表
output_notebook()
show(layout)


------有车场景------


Loading BokehJS ...

## 自车80kph，期望车速80kph，leader_v从80kph递减， leader_s 20m

In [29]:

print("------有车场景------")
cur_v = [80] * 20
desired_v = [80] * 20
obj_v = generate_arithmetic_list(80, -2, 20) # (first_element, difference, count)
delta_s = [20] * 20
idm_acc_list = []
delta_v_list = []
for i in range(len(cur_v)):
    idm_params.desired_spd = desired_v[i] / 3.6
    delta_v = (cur_v[i] - obj_v[i] ) / 3.6
    res = pybind_ilqr.LongitudinalOperator.calc_idm_acc(idm_params, delta_s[i], 
                                                        cur_v[i] / 3.6, delta_v, 1)
    idm_acc_list.append(res)
    delta_v_list.append(delta_v)
    # print(" has car, ", cur_v[i], " -> desired spd is:", desired_v[i],
    #       " , obj_v is:", obj_v[i], " , delta s is:", delta_s[i], " res:", res)
    # print("-------------------------------")
# pprint(idm_acc_list)

plot1 = figure(title='Leader_S-Acc', x_axis_label='leader_s', y_axis_label='acc',
                       plot_width=400, plot_height=300)
create_scatter_plot(plot1, delta_s, idm_acc_list, color='blue', legend_label='Leader_S-Idm_Acc')
# create_scatter_plot(plot1, delta_s, acc_crash_list, color='red', legend_label='Leader_S-Crash_Acc')
plot1.legend.click_policy = 'hide'
plot2 = figure(title='Leader_V-Acc', x_axis_label='leader_v', y_axis_label='acc',
                       plot_width=400, plot_height=300)
create_scatter_plot(plot2, obj_v, idm_acc_list, color='blue', legend_label='Leader_V-Idm_Acc')
# create_scatter_plot(plot2, obj_v, acc_crash_list, color='red', legend_label='Leader_V-Crash_Acc')
plot2.legend.click_policy = 'hide'
plot3 = plot_with_color_mapping(delta_s, obj_v, idm_acc_list, plot_width=400, plot_height=300)

layout = row(plot1, plot2, plot3)
output_notebook()
show(layout)


------有车场景------


Loading BokehJS ...

## 自车80kph，期望车速80kph，leader_v从60kph递减， leader_s从10m递增

In [19]:

print("------有车场景------")
cur_v = [80] * 20
desired_v = [80] * 20
obj_v = generate_arithmetic_list(60, -2, 20)  # (first_element, difference, count)
delta_s = generate_arithmetic_list(10, 2, 20)  # (first_element, difference, count)
idm_acc_list = []
delta_v_list = []
for i in range(len(cur_v)):
    idm_params.desired_spd = desired_v[i] / 3.6
    delta_v = (cur_v[i] - obj_v[i] ) / 3.6
    res = pybind_ilqr.LongitudinalOperator.calc_idm_acc(idm_params, delta_s[i], 
                                                        cur_v[i] / 3.6, delta_v, 1)
    idm_acc_list.append(res)
    delta_v_list.append(delta_v)
    # print(" has car, ", cur_v[i], " -> desired spd is:", desired_v[i],
    #       " , obj_v is:", obj_v[i], " , delta s is:", delta_s[i], " res:", res)
    # print("-------------------------------")
# pprint(idm_acc_list)

plot1 = figure(title='Leader_S-Acc', x_axis_label='leader_s', y_axis_label='acc',
                       plot_width=400, plot_height=300)
create_scatter_plot(plot1, delta_s, idm_acc_list, color='blue', legend_label='Leader_S-Idm_Acc')
# create_scatter_plot(plot1, delta_s, acc_crash_list, color='red', legend_label='Leader_S-Crash_Acc')
plot1.legend.click_policy = 'hide'
plot2 = figure(title='Leader_V-Acc', x_axis_label='leader_v', y_axis_label='acc',
                       plot_width=400, plot_height=300)
create_scatter_plot(plot2, obj_v, idm_acc_list, color='blue', legend_label='Leader_V-Idm_Acc')
# create_scatter_plot(plot2, obj_v, acc_crash_list, color='red', legend_label='Leader_V-Crash_Acc')
plot2.legend.click_policy = 'hide'
plot3 = plot_with_color_mapping(delta_s, obj_v, idm_acc_list, plot_width=400, plot_height=300)

layout = row(plot1, plot2, plot3)
output_notebook()
show(layout)


------有车场景------


Loading BokehJS ...

## 自车80kph，期望车速80kph，leader_v从20kph递增加， leader_s从10m递增

In [20]:

print("------有车场景------")
cur_v = [80] * 20
desired_v = [80] * 20
obj_v = generate_arithmetic_list(20, 2, 20)  # (first_element, difference, count)
delta_s = generate_arithmetic_list(10, 2, 20)  # (first_element, difference, count)
idm_acc_list = []
delta_v_list = []
for i in range(len(cur_v)):
    idm_params.desired_spd = desired_v[i] / 3.6
    delta_v = (cur_v[i] - obj_v[i] ) / 3.6
    res = pybind_ilqr.LongitudinalOperator.calc_idm_acc(idm_params, delta_s[i], 
                                                        cur_v[i] / 3.6, delta_v, 1)
    idm_acc_list.append(res)
    delta_v_list.append(delta_v)
    # print(" has car, ", cur_v[i], " -> desired spd is:", desired_v[i],
    #       " , obj_v is:", obj_v[i], " , delta s is:", delta_s[i], " res:", res)
    # print("-------------------------------")
# pprint(idm_acc_list)

plot1 = figure(title='Leader_S-Acc', x_axis_label='leader_s', y_axis_label='acc',
                       plot_width=400, plot_height=300)
create_scatter_plot(plot1, delta_s, idm_acc_list, color='blue', legend_label='Leader_S-Idm_Acc')
# create_scatter_plot(plot1, delta_s, acc_crash_list, color='red', legend_label='Leader_S-Crash_Acc')
plot1.legend.click_policy = 'hide'
plot2 = figure(title='Leader_V-Acc', x_axis_label='leader_v', y_axis_label='acc',
                       plot_width=400, plot_height=300)
create_scatter_plot(plot2, obj_v, idm_acc_list, color='blue', legend_label='Leader_V-Idm_Acc')
# create_scatter_plot(plot2, obj_v, acc_crash_list, color='red', legend_label='Leader_V-Crash_Acc')
plot2.legend.click_policy = 'hide'
plot3 = plot_with_color_mapping(delta_s, obj_v, idm_acc_list, plot_width=400, plot_height=300)

layout = row(plot1, plot2, plot3)
output_notebook()
show(layout)


------有车场景------


Loading BokehJS ...

# plot test

In [21]:
leader_s = generate_arithmetic_list(1.1, 1, 100)  # leader_s
leader_v = generate_arithmetic_list(1, 1, 100)  # leader_v
cur_v = generate_arithmetic_list(80, 0, 100)
desired_v = generate_arithmetic_list(80, 0, 100)
delta_v = [(x - y) / 3.6 for x, y in zip(cur_v, leader_v)]
idm_acc_list = []
acc_crash_list = []

for xi in leader_s:
    for yi in delta_v:
        idm_params.desired_spd = 80 / 3.6
        res = pybind_ilqr.LongitudinalOperator.calc_idm_acc(idm_params, xi, 80 / 3.6, yi, 1)
        # idm_acc_list.append(str(round(res, 1)))
        idm_acc_list.append(round(res, 1))
        acc_crash = (((80/3.6) - yi) ** 2 - (80 / 3.6) ** 2) / (2 * (xi - 1))
        acc_crash_list.append(acc_crash)

leader_s = generate_arithmetic_list(1, 1, 100)  # leader_s
leader_v = generate_arithmetic_list(1, 1, 100)  # leader_v
cur_v = generate_arithmetic_list(80, 0, 100)
delta_v = [(x - y) / 3.6 for x, y in zip(cur_v, leader_v)]

# 计算加速度
xx, yy, accelerations = [], [], []
for xi in leader_s:
    for yi in delta_v:
        # acc_crash = (yi ** 2 - v0 ** 2) / (2 * xi)
        acc_idm = pybind_ilqr.LongitudinalOperator.calc_idm_acc(idm_params, xi, 80 / 3.6, yi, 1, 0)
        xx.append(xi)
        yy.append(yi)
        accelerations.append(round(acc_idm, 1))
# 找到加速度为-4的点
acc_minus_four_x = [x for x, acc in zip(xx, accelerations) if acc == -2]
acc_minus_four_y = [y for y, acc in zip(yy, accelerations) if acc == -2]

# 创建数据源
source = ColumnDataSource(data=dict(x=xx, y=yy, acceleration=accelerations))

# 配置颜色映射
mapper = linear_cmap(field_name='acceleration', palette=Viridis256, low=np.min(accelerations), high=np.max(accelerations))
# color_mapper = linear_cmap(field_name='dist', palette=Viridis256, low=np.min(dist), high=np.max(dist))
# 创建图表
p = figure(title="Acceleration for Given Distance and Velocity")
p.scatter('x', 'y', color=mapper, fill_alpha=0.7, radius = 0.5, legend_label="Acceleration (m/s²)", source=source)
# 绘制加速度为-4的点的红色连接线
p.line(acc_minus_four_x, acc_minus_four_y, color="black", line_width=10)

# 创建并添加颜色条
color_mapper = LinearColorMapper(palette='Turbo256', low=min(accelerations), high=max(accelerations))
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0))
p.add_layout(color_bar, 'right')

show(p)
